EXERCISE 2

Imports

In [26]:
import cv2 as cv
import numpy as np

Reading Image

In [27]:
image = cv.imread("lena_grayscale_hq.jpg",0)

In [28]:
filter_size = 3 # this means (filter_size x filter_size filter.)

Necessary functions :

In [29]:
# creating our box filter and coefficient.
def create_filter(filter_size): 
      filter = np.ones( (filter_size,filter_size))
      zero_padding = determine_zero_padding_number(filter_size)
      return filter, filter_size* filter_size, zero_padding
# to determine how many zeros to add for zero padding for one pixel to a row or column.
def determine_zero_padding_number(filter_size): 
      return  ((int)(filter_size / 2 ))
# creating our new matrix to imply our box filter
def new_matrix_with_zeros(image,zero_padding_number):
      image_row, image_column = image.shape
      new_m = image_row + zero_padding_number * 2
      new_n = image_column + zero_padding_number * 2
      new_matrix = np.zeros( (new_m,new_n)  )
      new_matrix[zero_padding_number: zero_padding_number + image_row, zero_padding_number: zero_padding_number + image_column] = image
      return new_matrix
# filtering functions

# i,j centered piece taken.
def take_a_piece(big_matrix,i,j,padding_size):
      x1 = i-padding_size
      x2 = i+padding_size
      x3 = j-padding_size
      x4 = j+padding_size
      piece = big_matrix[x1:x2+1,x3:x4+1]
      return piece

def implement_filter(piece , box_filter,filter_coefficient):
      sum = np.sum(piece)
      pixel_value =  (sum /filter_coefficient)
      return pixel_value

def filtering(big_matrix, image, padding, filter_coefficient, box_filter):
      big_row, big_col = big_matrix.shape
      im_row, im_col = image.shape
      filtered_image = np.ones((im_row, im_col))
      for i in range(padding,padding + im_row):
            for j in range(padding, padding + im_col):
                  # dogru range'de doğru kez çalışıyor.
                  piece = take_a_piece(big_matrix,i,j,padding)
                  pixel_value = implement_filter(piece,box_filter,filter_coefficient)
                  filtered_image[i-padding,j-padding] = pixel_value
      return filtered_image           
                  
                                   
# image displaying function
def draw_images(image1, image2):
      cv.namedWindow("image1", cv.WINDOW_NORMAL)
      cv.namedWindow("image2", cv.WINDOW_NORMAL)
      cv.imshow("image1", image1)
      cv.imshow("image2", image2)
      cv.waitKey(0)
      cv.destroyAllWindows()     

def open_cv_implementation(image,filter_size):
      box_filter = np.ones((filter_size, filter_size), np.float32) / (filter_size * filter_size)
      border_type = cv.BORDER_CONSTANT
      opencv_filtered_image = cv.filter2D(image, -1, box_filter, borderType=border_type)
      return opencv_filtered_image

def my_implementation_of_blurring(image,filter_size):
      box_filter, filter_coefficient, padding = create_filter(filter_size)
      big_matrix = new_matrix_with_zeros(image,padding)
      filtered_matrix = filtering(big_matrix,image,padding,filter_coefficient,box_filter)
      filtered_matrix = np.uint8(np.round(filtered_matrix))
      return filtered_matrix

My Implementation :

3x3 - 11x11 - 21x21

In [30]:
output_1_1 = my_implementation_of_blurring(image,3)
output_1_2 = my_implementation_of_blurring(image,11)
output_1_3 = my_implementation_of_blurring(image,21)

OpenCV's Implementation

3x3 - 11x11 - 21x21

In [31]:
output_2_1 = open_cv_implementation(image,3)
output_2_2 = open_cv_implementation(image,11)
output_2_3 = open_cv_implementation(image,21)


Q1 - Evaluation Part :

In [32]:
difference_1 = np.sum(abs(output_1_1 - output_2_1 ))
difference_2 = np.sum(abs(output_1_2 - output_2_2 ))
difference_3 = np.sum(abs(output_1_3 - output_2_3 ))
print("3x3 difference is : ", difference_1)
print("11x11 difference is : ", difference_2)
print("21x21 difference is : ", difference_3)

3x3 difference is :  0
11x11 difference is :  0
21x21 difference is :  0


In [33]:
draw_images(output_1_1,output_2_1)
draw_images(output_1_2,output_2_2)
draw_images(output_1_3,output_2_3)

Q2 - Seperable Box Filter Implementation:

In [48]:
def seperable_1D_filter(filter_size):
   filter = np.ones( (1,filter_size) )
   padding = determine_zero_padding_number(filter_size)
   return filter, filter_size, padding 

def filtering_1D(big_matrix, image, padding, filter_coefficient, filter,is_transpoze):
   if is_transpoze == False :
      big_row, big_col = big_matrix.shape
      im_row, im_col = image.shape
      filtered_image = np.ones((im_row, im_col))
      for i in range(padding,padding + im_row):
            for j in range(padding, padding + im_col):
                  # dogru range'de doğru kez çalışıyor.
                  piece = take_1D_piece(big_matrix,i,j,padding,False)
                  pixel_value = implement_filter(piece,filter,filter_coefficient)
                  filtered_image[i-padding,j-padding] = pixel_value
      big_matrix[padding:padding+im_row, padding:padding + im_col] = filtered_image
      return big_matrix
   
   if is_transpoze == True :
      big_row, big_col = big_matrix.shape
      im_row, im_col = image.shape
      filtered_image = np.ones((im_row, im_col))
      for i in range(padding,padding + im_row):
            for j in range(padding, padding + im_col):
                  # dogru range'de doğru kez çalışıyor.
                  piece = take_1D_piece(big_matrix,i,j,padding,True)
                  pixel_value = implement_filter(piece,filter,filter_coefficient)
                  filtered_image[i-padding,j-padding] = pixel_value
      return filtered_image   
      

def take_1D_piece(big_matrix,i,j,padding, is_transpoze):
   if is_transpoze == False :
      x1 = j - padding
      x2 = j + padding
      piece = big_matrix[i,x1:x2+1]
   if is_transpoze == True :
      x1 = i - padding
      x2 = i + padding
      piece = big_matrix[x1:x2+1,j] 
   return piece
def implement_1D_filter(piece , filter_1D , filter_coefficient):
   sum = np.sum(piece)
   pixel_val = sum / filter_coefficient
   return pixel_val
def my_implementation_of_blurring_with_seperable_filter(image,filter_size):
   filter, filter_coefficient, padding = seperable_1D_filter(filter_size)
   big_matrix = new_matrix_with_zeros(image,padding)
   filtered_image = filtering_1D(big_matrix, image, padding, filter_coefficient, filter,False)
   filtered_image = filtering_1D(big_matrix, image, padding, filter_coefficient, filter,True)
   filtered_image = np.uint8(np.round(filtered_image))
   return filtered_image

My Implementation of Seperable Box Filter : 3x3 - 11x11 - 21x21

In [49]:
output_3_1 = my_implementation_of_blurring_with_seperable_filter(image,3)
output_3_2 = my_implementation_of_blurring_with_seperable_filter(image,11)
output_3_3 = my_implementation_of_blurring_with_seperable_filter(image,21)

Q2 - Evaluation Part :

In [50]:
difference_4 = np.sum(abs(output_3_1 - output_2_1 ))
difference_5 = np.sum(abs(output_3_2 - output_2_2 ))
difference_6 = np.sum(abs(output_3_3 - output_2_3 ))
print("3x3 difference is : ", difference_4)
print("11x11 difference is : ", difference_5)
print("21x21 difference is : ", difference_6)

3x3 difference is :  0
11x11 difference is :  0
21x21 difference is :  0


In [51]:
draw_images(output_3_1,output_2_1)
draw_images(output_3_2,output_2_2)
draw_images(output_3_3,output_2_3)